***
# Classification
MSDS 7331-407, Lab 2 
*Jenna Ford, Edward Fry, Christian Nava, and Jonathan Tan* 
***

## Table of Contents

<a href='#Section_1'> 1. Introduction </a>  
<a href='#Section_2'> 2. Preparation and Dataset Loading </a>  
<a href='#Section_3'> 3. Data Preparation Part 1 </a> 
<a href='#Section_4'> 4. Data Preparation Part 2 </a>  
<a href='#Section_5'> 5. Modeling and Evaluation 1 </a>  
<a href='#Section_6'> 6. Modeling and Evaluation 2 </a>  
<a href='#Section_7'> 7. Modeling and Evaluation 3 </a>  
<a href='#Section_7_a'> &nbsp;&nbsp;&nbsp;&nbsp; a. Arrest Type Code </a>  
<a href='#Section_7_b'> &nbsp;&nbsp;&nbsp;&nbsp; b. Descent Code </a>  
<a href='#Section_8'> 8. Modeling and Evaluation 4 </a>  
<a href='#Section_9'> 9. Modeling and Evaluation 5 </a>  
<a href='#Section_10'> 10. Modeling and Evaluation 6 </a>  
<a href='#Section_11'> 11. Deployment </a>  
<a href='#Section_12'> 12. Exceptional Work </a>  

<!-- <a href='#Section_4_c_i'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; i. Cross Street </a>    --> 

<a id = 'Section_1'></a>

## 1. Introduction

For this project our group will continue to use the Los Angeles arrest incidents dataset with arrest incidents dating back to 2010, which can be found [here](https://data.lacity.org/A-Safe-City/Arrest-Data-from-2010-to-Present/yru6-6re4). This dataset contains information about the date, time and location of the incident, demographic data of the person arrested, and information about the type of incident. 

We will be classifying the variables `Arrest Type` (Felony, Misdemeanor, Infraction, Other) and `Descent Code` (Black, Hispanic, White, Other). We feel that being able to classify arrest type could help in prioritizing dispatch calls, especially when there are more calls than officers available to respond. We assume that arrest type is an indicator of the severity of the incident. It is also worth noting that most of the data available for the classification would be available to the dispatcher. For example, someone placing a call to 911 would most likely have a basic description of the offender such as gender, approximate age and ethnicity. The caller would also know the location of the incident and have a brief description of the situation. 

As far as classifying `Descent Code`, we are not trying to make any political statements. However, if this variable can be predicted, what can that tell us? Any information discovered from this could be invetigated further. The information could be used for community outreach programs to help guide and direct efforts where they will make the most impact.

We 10-fold cross validation to test a variety of models. We also adjust model parameters to fine-tune our classification models. 

Model effectiveness will be measured using accuracy. A successful model would ideally exceed the ability of a dispatcher to correctly identify the arrest type. However, we do not have data available for this. To be considered effective, the model should significantly outperform random chance (25%), so we will use accuracy of 75% as the cut-off for identifying if the model is successful.

NEED TO WORK ON MORE AFTER MODELS AND EVALUATION METRICS ARE DETERMINED

<a id = 'Section_2'></a>

## 2. Preparation and Dataset Loading

In [ ]:
# Time the run
import time
startTime = time.time()

# Data manipulation
import numpy as np
import pandas as pd

# Creating training and test sets
import sklearn

# File system management
import os.path
from multiprocessing import Process
processes = []

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

#training/test split
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, ShuffleSplit

# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt

from sklearn.decomposition import PCA 
from sklearn.pipeline import Pipeline

# here we can change some of the parameters interactively
from ipywidgets import widgets as wd
from sklearn.model_selection import cross_val_score, cross_validate

#for weights standardization
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, mean_squared_error, classification_report

from matplotlib import pyplot as plt

# Support vector machines
from sklearn.svm import SVC

import plotly
plotly.offline.init_notebook_mode() # run at the start of every notebook

In [ ]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

In [ ]:
# Constants
DATAPATH_BASE = 'https://machinelearningi.blob.core.windows.net/group-project/'
DATAPATH_SAS_TOKEN = '?sv=2019-02-02&ss=bfqt&srt=sco&sp=rwdlacup&se=2020-04-27T11:12:37Z&st=2020-01-23T04:12:37Z&spr=https&sig=jpIpjrp8dIg9eyUyPpmgTe5yj9i1ZoCSru5kBVHcUO8%3D'
DATAPATH_FILENAME = 'Arrest_Data_from_2010_to_Present.csv'
DATAPATH_SMALL_FILENAME = 'Arrest_Data_from_2010_to_Present_Small.csv'

# Performance controls
SUB_SAMPLE_SIZE = 0.05  #subsample of dataframe
SVM_SUB_SAMPLE_SIZE = .05 #subsample for SVM, don't go past 15% of the data because it won't run
RUN_SVM1 = False
RUN_SVM2 = False
RUN_SVM3 = False
RUN_SVM4 = False
RUN_SVM5 = False
RUN_SVM6 = False
RUN_SVM7 = False
RUN_SVM8 = False
RUN_RF1 = False
RUN_RF2 = False
RUN_RF3 = False
RUN_RF4 = False
RUN_GRIDSEARCH = False
RUN_MAIN1 = False
RUN_MAIN2 = False

# Fully qualified paths ready to use
DATA_SOURCE = "".join([DATAPATH_BASE, DATAPATH_FILENAME, DATAPATH_SAS_TOKEN])
#DATA_SOURCE = '../Arrest_Data_from_2010_to_Present.csv'

# Options
pd.set_option('float_format', '{:.2f}'.format)  # Reign in the scientific notation for reasonable values

# Load data for analysis; only read if needed because the import can take a long time
try:
    if len(df.index) < 1:
        df_raw = pd.read_csv(DATA_SOURCE) # If we get here, the dataframe was empty
except:   
    df_raw = pd.read_csv(DATA_SOURCE) # If we get here, the dataframe did not exist
    

In [ ]:
df = df_raw

print("The dataset has {:,} rows and {:,} columns".format(*df.shape))

<a id = 'Section_3'></a>

## 3. Data Preparation Part 1

*(10 points)*  

*Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.*

The code below is from Lab 1 where we prepared the initial dataset. Here is a summary of the steps in the code below:
* Convert `Time` to a time format and treat missing values and a value of 24:00 as 00:00.
* Drop observations where the age of the individual is under 16. This is due to data issues (probably data entry errors).
* Drop observations where the `Arrest Type Code` is Dependent. The majority of these observations are for individuals aged 15 or less (and have already been removed from the dataset). There are, however, some remaining observations that we believe to be data entry errors.
* Reduce the `Descent Code` classifications to B-Black, H-Hispanic and W-White. Any other ethnicities are grouped into O-Other.
* Create an `Hour` variable because knowing the hour and minute an incident occurred is not critical. A window of time will be more appropriate.
* Create variables for the day of week and month an incident occurred on. Knowing the exact date an incident occurred is not helpful for future classification, but day of week and month may be.
* Drop variables that are not useful for the analysis such as `Location` which has GIS coordinates and description variables that have an associated code variable.

In [ ]:
# Time - filter out 0 and missing
df = df[df['Time'] != 0]
df['Time'] = df['Time'].astype(str) 
df = df[df['Time'] != 'nan']

# Time - Convert float to string. Get rid of decimals. Replace missing or invalid values with '0000'.
df['Time'] = df['Time'].astype(str).str.split(".", expand = True)[0].replace(to_replace = ['2400','nan'], value = '0000') 

# Time - Fill time column with leading zeros to have 4 characters total
df['Time'] = df['Time'].apply(lambda x: '{0:0>4}'.format(x))

# Time - Add colon to Time values by converting attribute to a datetime variable 
df['Time'] = pd.to_datetime(df['Time'], format = '%H%M').dt.time

# Charge Group Code - filter out missing values
df = df.loc[df['Charge Group Code'].notnull()]

# Age - Drop the observations where Age is less than 16
df.drop(df[df['Age'] < 16].index, inplace = True) 

# Arrest Type Code - Drop the observations where Arrest Type Code = 'D'
df.drop(df[df['Arrest Type Code'] == 'D'].index, inplace = True) 

# Descent Code - Re-classify any descent not in (B,H,O,W) into 0
descent_list = ['B','H','O','W']
df['Descent Code'] = np.where(np.isin(df['Descent Code'],descent_list),df['Descent Code'],'O')

# Get hour
df['Hour'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.hour

# Convert Arrest Date to datetime
df['Arrest Date'] = pd.to_datetime(df['Arrest Date'])

# Extract month, and day of week and add to dataframe as new attributes
df['arrest_month']= df['Arrest Date'].dt.month
try:
    df['arrest_day_of_week'] = df['Arrest Date'].dt.weekday_name
except:
    df['arrest_day_of_week'] = 'monday'

# remove unecessary columns
df.drop(['Cross Street','Charge Description','Charge','Charge Group Description','Time',
         'Arrest Date','Report ID','Address','Area Name','Location'], axis=1, inplace=True)

# Change data types
df['Age'] = df['Age'].astype(np.int8)
df['Reporting District'] = df['Reporting District'].astype(np.str)
df['Area ID'] = df['Area ID'].astype(np.str)
df['Charge Group Code'] = df['Charge Group Code'].astype(np.str)
df['Hour'] = df['Hour'].astype(np.str)
df['arrest_month'] = df['arrest_month'].astype(np.str)
df['arrest_day_of_week'] = df['arrest_day_of_week'].astype(np.str)

df_lightgbm = df
# print clean dataset
df.info()

The code below prepares the dataset for classification tasks. First, we group `Age` into bins of 10 years and perform integer encoding, since the data is ordinal. Next, we perform one-hot encoding on all categorical variables. We use one-hot encoding here because no natural ordering exists in the values.

In [ ]:
# Create buckets for Age
df['age_range'] = pd.cut(df.Age,[15,25,35,45,55,65,75,1e6],4,labels=[0,1,2,3,4,5,6]) # this creates a new variable
df['age_range'] = df['age_range'].fillna(0)
df['age_range'] = df.age_range.astype(np.int)

# Replace the current Sex atribute with something slightly more intuitive and readable
df['IsMale'] = df['Sex Code']=='M' 
df.IsMale = df.IsMale.astype(np.int)

# Perform one-hot encoding of the categorical data "DOW"
tmp_df = pd.get_dummies(df['arrest_day_of_week'],prefix='DOW',drop_first=True)
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

# Perform one-hot encoding of the categorical data "Area ID"
tmp_df = pd.get_dummies(df['Area ID'],prefix='Area',drop_first=True)
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

# Perform one-hot encoding of the categorical data "Charge Group Code"
tmp_df = pd.get_dummies(df['Charge Group Code'],prefix='Charge',drop_first=True)
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

# Perform one-hot encoding of the categorical data "Hour"
tmp_df = pd.get_dummies(df['Hour'],prefix='Hour',drop_first=True)
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

# Perform one-hot encoding of the categorical data "Month"
tmp_df = pd.get_dummies(df['arrest_month'],prefix='Month',drop_first=True)
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

Since we have 2 classification tasks (`Arrest Type Code` and `Descent Code`), we need 2 datasets. The dataset for classifying `Arrest Type Code` will need to have `Descent Code` one-hot encoded and the dataset for classifying `Descent Code` will need to have `Arrest Type Code` one-hot encoded.

In [ ]:
df_arrest = df
df_descent = df

#Final encoding steps for Arrest Type Code classification dataset
# Encode Arrest Type Code as Categorical
cleanup_arrest = {"Arrest Type Code": {"F": 0, "M": 1, "I": 2, "O":3}}
df_arrest.replace(cleanup_arrest,inplace=True)

# Perform one-hot encoding of the categorical data "Descent Code"
tmp_df = pd.get_dummies(df_arrest['Descent Code'],prefix='Descent',drop_first=True)
df_arrest = pd.concat((df_arrest,tmp_df),axis=1) # add back into the dataframe

df_arrest.drop(['Sex Code','Descent Code','arrest_day_of_week','Area ID','Reporting District','Charge Group Code',
         'Age','Hour','arrest_month'], axis=1, inplace=True)

df_arrest.info()

In [ ]:
#Final encoding steps for Descent Code classification dataset
cleanup_descent = {"Descent Code": {"B": 0, "H": 1, "W": 2, "O":3}}
df_descent.replace(cleanup_descent,inplace=True)

# Perform one-hot encoding of the categorical data "Arrest Type Code"
tmp_df = pd.get_dummies(df_descent['Arrest Type Code'],prefix='Arrest',drop_first=True)
df_descent = pd.concat((df_descent,tmp_df),axis=1) #add back to the dataframe

df_descent.drop(['Sex Code','Arrest Type Code','arrest_day_of_week','Area ID','Reporting District','Charge Group Code',
         'Age','Hour','arrest_month'], axis=1, inplace=True)

df_descent.info()

In [ ]:
print("The Arrest classification dataset set has {:,} rows and {:,} columns".format(*df_arrest.shape))
print("The Descent classification dataset set has {:,} rows and {:,} columns".format(*df_descent.shape))

<a id = 'Section_4'></a>

## 4. Data Preparation Part 2

*(5 points)*

*Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).*

#### Table 1: Description of Final Dataset

| Dataset | Target/Explanatory| Variable Prefix | Variable Suffix Range | Description |
| :-- | :-- | :-- | :-- | :-- |
| Arrest | Target| Arrest Type Code | NA | Felony=0, Misdemeanor=1, Infraction=2, Other=3 |  
| Descent | Target| Descent Code | NA | Black=0, Hispanic=1, White=2, Other=3 | 
| Both | Explanatory| age_range | NA | Age bins: 16-25=0, 25-35=1, 35-45=2, 45-55=3, 55-65=4, 65-75=5, 75+=6|
| Both | Explanatory| IsMale | NA | 0=Female, 1=Male|
| Both | Explanatory| DOW | Saturday-Thursday | Value of 1 indicates the DOW the incident occurred on. DOW_Friday is the comparison and was dropped from the dataset.|
| Both | Explanatory| Area | 2-21 | Value of 1 indicates the Area the incident occurred in. Area_1 is the comparison and was dropped from the dataset.|
| Both | Explanatory| Charge | 2-29, 99, nan | Value of 1 indicates the Charge relating to the incident. Charge_1 is the comparison and was dropped from the dataset.|
| Both | Explanatory| Hour | 1-23 | Value of 1 indicates the Hour the incident occurred. Hour_0 (00:00-00:59 on the 24-hour clock) is the comparison and was dropped from the dataset.|
| Both | Explanatory| Month | 2-12 | Value of 1 indicates the month the incident occurred. Month_1 (January) is the comparison and was dropped from the dataset.|
| Arrest | Explanatory| Descent | H,W,O | Value of 1 indicates the descent of the individual. Descent_H: Hispanic, Descent_W: White and Descent_O: Other. Descent_B (Black) is the comparison and was dropped from the dataset.|
| Descent | Explanatory| Arrest | 1-3 | Value of 1 indicates the type of arrest made. Arrest_1: Misdemeanor, Arrest2: Infraction and Arrest_3: Other. Arrest_0 (Felony) is the comparison and was dropped from the dataset.|

<a id = 'Section_5'></a>

## 5. Modeling and Evaluation 1

*(10 points)*

*Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.*

The metric used for this project is the area under the receiver operating characteristic curve (AUC). The receiver operating characteristic curve (ROC) graphs the performance of a classification model at all classification thresholds; it plots the true positive rate (TPR) against the false positive rate (FPR) as shown in Figure 1.

<h4 align="center">Figure 1: Area Under the Reciever Operating Curve</h4> 
<img src="https://marlin-prod.literatumonline.com/cms/attachment/34661288-1f8f-459e-b8b4-936efc49e9bc/fx1_lrg.jpg" style="width:350px;height:300px"/>

Source: [Journal of Thoracic and Cardiovascular Surgery](https://www.jtcvs.org/article/S0022-5223(18)32875-7/fulltext)

Each curve represents the graph of a single model. Movement along the curve indicates changing the threshold used for classifying a positive instance. The area under the ROC curve measures the two-dimensional area under the ROC curve and is given by the integral of the curve, or the area under the curve (AUC): $$AUC = \int_{0}^{1}TPR(x)dx = \int_{0}^{1}P(A>\tau (x))dx$$

where $x$ is the false positive rate and $A$ is the distribution of scores the model produces for data points that are actually in the positive class.

This metric is measured between 0 and 1. A higher score indicates a better model. Visually, a superior model will graph as a curve above and to the left of another curve. A straight diagonal line beginning at 0 on the bottom left and ending at 1 on the top right indicates a naive model (random guessing). Other metrics such as accuracy are not the best metric for data with imbalanced classes. The Los Angeles City Arrest dataset is imbalanced. 

A model with a high ROC AUC will also have a high accuracy, therefore, the AUC score will be the metric employed in this analysis.


<a id = 'Section_6'></a>

## 6. Modeling and Evaluation 2

*(10 points)*

*Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.*

Cross validation allows us to test our model on data it hasn't seen, to see how well the model performs when making predictions. Using cross validation can help identify if the model is overfitting the training data. If the model is overfitting the training data we would expect to see low evaluation metrics when predictions are created for the test dataset.

We use stratified 10-fold cross validation for this project. The classifiers we are using for this project are unbalanced. The first table below shows that 61% of arrests are Misdemeanors and only 2% of arrests are identified as Other. The second table below shows that 46% of arrests are for Hispanic individuals and 6% of arrests are for individuals grouped into an Other ethnicity bin. Balanced data would have approximately 25% of all arrests in each bucket (for both classifiers).

<a id = 'Section_7'></a>

In [ ]:
#create a function to count by column and display percentages
def count_percent(data,field):
    df_grouped = data.groupby(by=field)
    c1 = (df_grouped[field].count())

    c2  = []#create empty list to store percentages
    for x in c1:
        c2.append("{:.2%}".format((x/sum(c1)))) #row value divided by total, formatted as percent, store in list
    c1 = pd.DataFrame(c1) #needs dataframe to start with, then can add new column from list c2
    c1['Percent']= c2
    c1.columns = ['Count','Percent']
    return c1

In [ ]:
count_percent(df,'Arrest Type Code')

In [ ]:
count_percent(df,'Descent Code')

Due to having extremely unbalanced data it is important for us to use a cross validation technique that stratifies the sample, matching the same percentages of the classifier from the population dataset. This is the 'stratified' part of the stratified 10-fold cross validation tecnhique we have chosen. Without stratifying the folds we cannot ensure that the folds are representative of the population; leading to high bias and variance.

When splitting data into training and test datasets there is a concern that important patterns found in the data may be left out of the training dataset. If these patterns are not present in the training dataset the model won't be able to predict them in the test dataset. Using k-fold cross validation takes this into account. K-fold cross validation divides the data into k folds. K-1 folds are used to train the model and the remaining fold is used for testing. This is repeated for k-1 models and every fold is used for testing once. This reduces bias and variance since we are essentially using the entire dataset for training and testing. We have chosen 10-folds for our project, a typical value chosen.

<a id = 'Section_7'></a>

## 7. Modeling and Evaluation 3

*(20 points)*

*Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!*

<a id = 'Section_7_a'></a>

In [ ]:
df_arrest1 = df_arrest
df_descent1 = df_descent

#subsample of dataframe
df_arrest1 = pd.DataFrame.sample(df_arrest1, frac = SUB_SAMPLE_SIZE, random_state = 34128)
df_descent1 = pd.DataFrame.sample(df_descent1, frac = SUB_SAMPLE_SIZE, random_state = 34128)

print("The datasets have {:,} rows and {:,} columns".format(*df_arrest1.shape))

### a. Arrest Type Code

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import metrics as mt


# create variables we are more familiar with
if 'Arrest Type Code' in df_arrest1:
    ya = df_arrest1['Arrest Type Code'].values # get the labels we want
    del df_arrest1['Arrest Type Code'] # get rid of the class label
    Xa = df_arrest1.values # use everything else to predict!
    
yhata = np.zeros(ya.shape) # we will fill this with predictions

scl = StandardScaler()
Xa = scl.fit_transform(Xa)

# create cross validation iterator
cv = StratifiedKFold(n_splits=10, random_state=1234)

In [ ]:
def per_class_accuracy(ytrue,yhat):
    conf = mt.confusion_matrix(ytrue,yhat)
    norm_conf = conf.astype('float') / conf.sum(axis=1)[:, np.newaxis]
    return np.diag(norm_conf)

def plot_class_acc(ytrue,yhat, title=''):
    acc_list = per_class_accuracy(ytrue,yhat)
    plt.bar(range(len(acc_list)), acc_list)
    plt.xlabel('0-Felony, 1-Misdemeanor, 2-Infraction, 3-Other')
    plt.ylabel('Accuracy within class')
    plt.title(title+", Total Acc=%.1f"%(100*mt.accuracy_score(ytrue,yhat)))
    plt.grid()
    plt.ylim([0,1])
    plt.show()

### XGBoost - Arrest Type Code

In [ ]:
#KGBoost 
import pickle
import xgboost as xgb

import numpy as np
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error, classification_report, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def XGBoosta(boost, tree, depth, delta_step, etaparm):
    for train, test in cv.split(Xa,ya):
        xgb_model = xgb.XGBClassifier(booster=boost, tree_method=tree, max_depth=depth, max_delta_step=delta_step,eta=etaparm).fit(Xa[train], ya[train])
        yhata[test] = xgb_model.predict(Xa[test])
    
    lb=LabelBinarizer()
    lb.fit(ya)
    ya_lb = lb.transform(ya)
    yhata_lb = lb.transform(yhata)
    print ('AUC Score', roc_auc_score(ya_lb, yhata_lb, average='weighted'))
    
    print (mt.classification_report(ya,yhata,digits=3))
                
    plot_class_acc(ya,yhata,title="XGBoost")

#### Defaults

In [ ]:
%%time
if RUN_MAIN1:
    boost='gbtree'
    tree='auto'
    depth=6
    delta_step=0
    etaparm=0.3

    XGBoosta(boost,tree,depth,delta_step,etaparm)

#### Modifying Booster Parameter

In [ ]:
%%time
if RUN_MAIN1:
    boost='gblinear'
    tree='auto'
    depth=6
    delta_step=0
    etaparm=0.3

    XGBoosta(boost,tree,depth,delta_step,etaparm)

In [ ]:
%%time
if RUN_MAIN1:
    boost='dart'
    tree='auto'
    depth=6
    delta_step=0
    etaparm=0.3

    XGBoosta(boost,tree,depth,delta_step,etaparm)

#### Modifying Booster & Tree Method Parameters

In [ ]:
%%time
if RUN_MAIN1:
    boost='gbtree'
    tree='hist'
    depth=6
    delta_step=0
    etaparm=0.3

    XGBoosta(boost,tree,depth,delta_step,etaparm)

In [ ]:
%%time
if RUN_MAIN1:
    boost='gblinear'
    tree='hist'
    depth=6
    delta_step=0
    etaparm=0.3

    XGBoosta(boost,tree,depth,delta_step,etaparm)

In [ ]:
%%time
if RUN_MAIN1:
    boost='dart'
    tree='hist'
    depth=6
    delta_step=0
    etaparm=0.3

    XGBoosta(boost,tree,depth,delta_step,etaparm)

#### Modifying Booster, Tree Method and Max Depth

In [ ]:
%%time
if RUN_MAIN1:
    boost='gbtree'
    tree='hist'
    depth=5
    delta_step=0
    etaparm=0.3

    XGBoosta(boost,tree,depth,delta_step,etaparm)

In [ ]:
%%time
if RUN_MAIN1:
    boost='gbtree'
    tree='hist'
    depth=7
    delta_step=0
    etaparm=0.3

    XGBoosta(boost,tree,depth,delta_step,etaparm)

#### Modifying Booster, Tree Method and Max Delta Step

In [ ]:
%%time
if RUN_MAIN1:
    boost='gbtree'
    tree='hist'
    depth=6
    delta_step=1
    etaparm=0.3

    XGBoosta(boost,tree,depth,delta_step,etaparm)

In [ ]:
%%time
if RUN_MAIN1:
    boost='gbtree'
    tree='hist'
    depth=6
    delta_step=5
    etaparm=0.3

    XGBoosta(boost,tree,depth,delta_step,etaparm)

In [ ]:
%%time
if RUN_MAIN1:
    boost='gbtree'
    tree='hist'
    depth=6
    delta_step=10
    etaparm=0.3

    XGBoosta(boost,tree,depth,delta_step,etaparm)

#### XGBoost Performance - Arrest Type Code

#### Table 2: XGBoost Performance for Arrest Type Code
Performance metrics for each Arrest Type are listed as *Precision, Recall*.

| Booster | Tree Method | Max Depth | Max Delta Step | Time | Accuracy | AUC | Felony | Misdemeanor | Infraction | Other | 
| :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- |
| gbtree | auto | 6 | 0 | | | | | | | |
| gblinear | auto | 6 | 0 | | | | | | | |
| dart | auto | 6 | 0 |  | | | | | | |
| gbtree | hist | 6 | 0 | | | | | | | |
| gblinear | hist | 6 | 0 | | | | | | | |
| dart | hist | 6 | 0 | | | | | | | |
| gbtree | hist | 5 | 0 | | | | | | | |
| gbtree | hist | 7 | 0 | | | | | | | |
| gbtree | hist | 6 | 1 | | | | | | | |
| gbtree | hist | 6 | 5 | | | | | | | |
| gbtree | hist | 6 | 10 | | | | | | | |

### LightGBM - Arrest Type Code

In [ ]:
import lightgbm as lgb


def LightGBMa(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample):
    for train, test in cv.split(Xa,ya):
        lgbm_model = lgb.LGBMClassifier(n_estimators = estimators,
                                        objective = 'multiclass',
                                        learning_rate = l_rate,
                                        num_leaves = leaves,
                                        lambda_l1 = r_alpha,
                                        lambda_l2 = r_lambda, 
                                        maxdepth = depth,
                                        bagging_fraction = subsample,
                                        nthread = thread,
                                        min_gain_to_split = split_gain,
                                        feature_fraction = colsample,
                                        random_state = 17).fit(Xa[train], ya[train])
        yhata[test] = lgbm_model.predict(Xa[test])
    
    lb = LabelBinarizer()
    lb.fit(ya)
    ya_lb = lb.transform(ya)
    yhata_lb = lb.transform(yhata)
    print ('AUC Score', roc_auc_score(ya_lb, yhata_lb, average = 'weighted'))
    
    print (mt.classification_report(ya, yhata, digits = 3))
                
    plot_class_acc(ya, yhata, title = "LightGBM")

In [ ]:
%%time
if RUN_MAIN1:
    estimators = 100
    objective = 'multiclass'
    l_rate = 0.1
    leaves = 31
    r_alpha = 0.0
    r_lambda = 0.0
    depth = -1
    subsample = 1.0
    thread = 0
    split_gain = 0.0
    colsample = 1.0


    LightGBMa(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN1:
    estimators = 1000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 31
    r_alpha = 0.0
    r_lambda = 0.0
    depth = -1
    subsample = 1.0
    thread = 0
    split_gain = 0.0
    colsample = 1.0


    LightGBMa(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN1:
    estimators = 10000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 31
    r_alpha = 0.0
    r_lambda = 0.0
    depth = -1
    subsample = 1.0
    thread = 0
    split_gain = 0.0
    colsample = 1.0


    LightGBMa(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN1:
    estimators = 1000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 31
    r_alpha = 0.5
    r_lambda = 0.5
    depth = 2
    subsample = 1.0
    thread = 0
    split_gain = 0.02
    colsample = 0.8


    LightGBMa(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN1:
    estimators = 10000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 31
    r_alpha = 0.5
    r_lambda = 0.5
    depth = 2
    subsample = 1.0
    thread = 0
    split_gain = 0.02
    colsample = 0.8


    LightGBMa(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN1:
    estimators = 10000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 31
    r_alpha = 0.05
    r_lambda = 0.05
    depth = 7
    subsample = 1.0
    thread = 0
    split_gain = 0.02
    colsample = 0.8


    LightGBMa(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN1:
    estimators = 10000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 34
    r_alpha = 0.05
    r_lambda = 0.05
    depth = 7
    subsample = 1.0
    thread = 0
    split_gain = 0.02
    colsample = 0.8


    LightGBMa(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

#### LightGBM Performance - Arrest Type Code

#### Table 3: LightGBM Performance for Arrest Type Code
Performance metrics for each Arrest Type are listed as *Precision, Recall*.

| Param | Param | Param | Param | Time | Accuracy | AUC | Felony | Misdemeanor | Infraction | Other | 
| :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- |
| gbtree | auto | 6 | 0 | | | | | | | |
| gblinear | auto | 6 | 0 | | | | | | | |
| dart | auto | 6 | 0 |  | | | | | | |
| gbtree | hist | 6 | 0 | | | | | | | |
| gblinear | hist | 6 | 0 | | | | | | | |
| dart | hist | 6 | 0 | | | | | | | |
| gbtree | hist | 5 | 0 | | | | | | | |
| gbtree | hist | 7 | 0 | | | | | | | |
| gbtree | hist | 6 | 1 | | | | | | | |
| gbtree | hist | 6 | 5 | | | | | | | |
| gbtree | hist | 6 | 10 | | | | | | | |

### Random Forest - Arrest Type Code

#### Random Forest Performance - Arrest Type Code

In [ ]:
%%time
if RUN_RF1:
    from sklearn.ensemble import RandomForestClassifier

    maxAccPCARF = 0
    optimalEstimatorsPCARF = 0
    optimalYPCARF = []
    optimalYhatPCARF = []
    clReport = ''

    for i in range(100, 201, 50):
        clf_pipe = Pipeline(
            [('PCA',PCA(n_components=50, svd_solver='randomized')),
             ('CLF',RandomForestClassifier(n_estimators=i, n_jobs=-1))]
        )

        # now iterate through and get predictions, saved to the correct row in yhat
        for train, test in cv.split(Xa,ya):
            clf_pipe.fit(Xa[train],ya[train])
            yhata[test] = clf_pipe.predict(Xa[test])

        plt.title('Confusion Matrix for {0} Estimators'.format(i), fontsize = 20)
        plt.imshow(mt.confusion_matrix(ya, yhata),cmap=plt.get_cmap('Greens'),aspect='auto')
        plt.grid(False)
        plt.show()

        total_accuracy = mt.accuracy_score(ya, yhata)
        if total_accuracy > maxAccPCARF:
            maxAccPCARF = total_accuracy
            optimalEstimatorsPCARF = i
            optimalYPCARF = ya
            optimalYhatPCARF = yhata
            clReport = mt.classification_report(ya,yhata,digits=3)

        lb = LabelBinarizer()
        lb.fit(ya)
        ya_lb = lb.transform(ya)
        yhata_lb = lb.transform(yhata)
        print ("Acc: ", total_accuracy)
        print ("AUC: ", roc_auc_score(ya_lb, yhata_lb))

    print ('Best accuracy is ', maxAccPCARF, ' with ', optimalEstimatorsPCARF, ' estimators in a random forest with PCA')
    print(clReport)
    plot_class_acc(optimalYPCARF,optimalYhatPCARF,title="Random Forest + PCA")

In [ ]:
%%time
if RUN_RF2:
    from sklearn.ensemble import RandomForestClassifier
    maxAccRF = 0
    optimalEstimatorsRF = 0
    clReport = ''

    for i in range(100, 201, 50):
        clf = RandomForestClassifier(n_estimators=i, n_jobs=-1, oob_score=True)

        # now iterate through and get predictions, saved to the correct row in yhat
        for train, test in cv.split(Xa,ya):
            clf.fit(Xa[train],ya[train])
            yhata[test] = clf.predict(Xa[test])

        total_accuracy = mt.accuracy_score(ya, yhata)

        plt.title('Confusion Matrix for {0} estimators'.format(i), fontsize = 20)
        plt.imshow(mt.confusion_matrix(ya, yhata),cmap=plt.get_cmap('Greens'),aspect='auto')
        plt.grid(False)
        plt.show()

        if total_accuracy > maxAccRF:
            maxAccRF = total_accuracy
            optimalEstimatorsRF = i
            optimalYRF = ya
            optimalYhatRF = yhata
            clReport = mt.classification_report(ya,yhata,digits=3)

        lb = LabelBinarizer()
        lb.fit(ya)
        ya_lb = lb.transform(ya)
        yhata_lb = lb.transform(yhata)
        print ("Acc: ", total_accuracy)
        print ("AUC: ", roc_auc_score(ya_lb, yhata_lb))

    print ('Best accuracy is ', maxAccRF, ' with ', optimalEstimatorsRF, ' Estimators in a Raw Random Forest')
    print(clReport)
    plot_class_acc(optimalYRF,optimalYhatRF,title="Random Forest, Raw")

#### Table 4: Random Forest Performance for Arrest Type Code
Performance metrics for each Arrest Type are listed as *Precision, Recall*.

| Param | Param | Param | Param | Time | Accuracy | AUC | Felony | Misdemeanor | Infraction | Other | 
| :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- |
| gbtree | auto | 6 | 0 | | | | | | | |
| gblinear | auto | 6 | 0 | | | | | | | |
| dart | auto | 6 | 0 |  | | | | | | |
| gbtree | hist | 6 | 0 | | | | | | | |
| gblinear | hist | 6 | 0 | | | | | | | |
| dart | hist | 6 | 0 | | | | | | | |
| gbtree | hist | 5 | 0 | | | | | | | |
| gbtree | hist | 7 | 0 | | | | | | | |
| gbtree | hist | 6 | 1 | | | | | | | |
| gbtree | hist | 6 | 5 | | | | | | | |
| gbtree | hist | 6 | 10 | | | | | | | |

### SVM - Arrest Type Code

In this section, we will explore support vector machines. First, a variety of SVM kernels are used, and we compare the results to see which ones are the most and least accurate. For the linear kernel, we can take the additional step of looking at the weights and plotting out the most meaningful.  Once we have the most accurate SVM model, which turns out to be RBF, we plot KDEs for all support vectors and compare that to their before state.  Finally, just for fun, we also look at a stochastic gradient descent (SGD) model. Interestingly, the SGD model performs about as well as the top models that don't use SGD.

One thing that we noticed right away is just how long it takes to train SVM models. We found that the training time grew exponentially as the size of the dataset grew, and we simply did not have enough time, after days of waiting, to use the full dataset in this section.  As a result, we opted to use a much smaller version of the dataset, which was built by randomly sampling 15% of data in the original dataset.  This resulted in 160,477 observations, which is much more manageable.

In [ ]:
#changing the sample size because we can't get the full dataset through SVM
df_arrest2 = df_arrest
df_descent2 = df_descent

#subsample of dataframe
df_arrest2 = pd.DataFrame.sample(df_arrest2, frac = SVM_SUB_SAMPLE_SIZE, random_state = 34128)
df_descent2 = pd.DataFrame.sample(df_descent2, frac = SVM_SUB_SAMPLE_SIZE, random_state = 34128)


print("The datasets have {:,} rows and {:,} columns".format(*df_arrest2.shape))

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import metrics as mt


# create variables we are more familiar with
if 'Arrest Type Code' in df_arrest2:
    yaS = df_arrest2['Arrest Type Code'].values # get the labels we want
    del df_arrest2['Arrest Type Code'] # get rid of the class label
    XaS = df_arrest2.values # use everything else to predict!
    
yhataS = np.zeros(yaS.shape) # we will fill this with predictions

scl = StandardScaler()
XaS = scl.fit_transform(XaS)

# create cross validation iterator
cv = StratifiedKFold(n_splits=10, random_state=1234)

In [ ]:
def SVMa(krnl):
    for train, test in cv.split(XaS,yaS):
        svm_clf = SVC(C=0.5, kernel=krnl, degree=3, gamma='auto').fit(XaS[train], yaS[train])
        yhataS[test] = svm_clf.predict(XaS[test])

    lb=LabelBinarizer()
    lb.fit(yaS)
    yaS_lb = lb.transform(yaS)
    yhataS_lb = lb.transform(yhataS)
    print ('AUC Score', roc_auc_score(yaS_lb, yhataS_lb, average='weighted'))

    print (mt.classification_report(yaS,yhataS,digits=3))

    plot_class_acc(yaS,yhataS,title="SVM")

#### SVM Sigmoid Kernel
Now, we can start our SVM analysis.  Here we train a model using a Sigmoid kernel.  The sigmoid is a special function that is guaranteed to have a y-value constrained between 0 and 1 and crosses the y-axis at x=0.  Accuracy was better than logistic regression, at 74.25%, but we can do better.  The confusion matrix shows our predicted results across the four arrest types - Felony, Misdemeanor, Infraction, and Other (in this order).  This gives us a 4x4 matrix with the accurately predicted counts along the main diagonal.

In [ ]:
%%time

if RUN_SVM1:
    krnl='sigmoid'
    SVMa(krnl)

#### SVM Linear Kernel

Note that linear is the simplest model to use conceptually, as it really is as simple as drawing a line between clusters of data points (or a plane in three dimensions, a cube in four, and so on).  The result is an accuracy of 76.86%, the highest yet.  In addition to the usual confusion matrix, we have also printed out the support vectors themselves and their n-value to study their shape for this dataset.

In [ ]:
%%time

if RUN_SVM2:
    krnl='linear'
    SVMa(krnl)

#### SVM Polynomial Kernel
Next, we use the polynomial kernel, which allows us to learn a non-linear model.  We can specify the degree (i.e. the highest number of exponents of the polynomial).  In this case, we chose to go with 3.  While higher order polynomials would result in better accuracy, they would likely overfit the model, thus making it a poor predictor.

Once fit, we look at the test results and find that this model results in about 81% accuracy.  This is significantly better than the accuracy results from logistic regression.

In [ ]:
%%time

if RUN_SVM3:
    krnl='poly'
    SVMa(krnl)

#### SVM Radial Basis Function Kernel
For our final SVM model, we use RBF, which, much like sigmoid, ranges between 0 and 1.  It measures the distance between points and the support vectors, and another useful property is that RBF decreases with distance.  The result is an accuracy 81.12%.

In [ ]:
%%time

if RUN_SVM4:
    krnl='rbf'
    SVMa(krnl)

#### Stochastic Gradient Descent
Just to see what the differences might be, we also tried a model with SGD.  Much like SVM, this also involved the use of some scaling before getting the test set prediction.  The results and confusion matrix show that accuracy with this method was as good as with SVM using and RBF kernel.  While performance in this analysis was very similar for both models, using SGD could result in better performance since it doesn't explode exponentially the way that the SVM algorithm does.

In [ ]:
%%time
# Linear SVM classifier with Stochastic Descent
if RUN_MAIN1:
    from sklearn.linear_model import SGDClassifier

    regularize_const = 0.1
    iterations = 5


    for train, test in cv.split(XaS,yaS):
        svm_clf = svm_sgd = SGDClassifier(alpha=regularize_const,
            fit_intercept=True, l1_ratio=0.0, learning_rate='optimal',
            loss='hinge', n_iter_no_change=iterations, n_jobs=-1, penalty='l2').fit(XaS[train], yaS[train])
        yhataS[test] = svm_clf.predict(XaS[test])

    lb=LabelBinarizer()
    lb.fit(yaS)
    yaS_lb = lb.transform(yaS)
    yhataS_lb = lb.transform(yhataS)
    print ('AUC Score', roc_auc_score(yaS_lb, yhataS_lb, average='weighted'))

    print (mt.classification_report(yaS,yhataS,digits=3))

    plot_class_acc(yaS,yhataS,title="SGD")

#### SVM Performance - Arrest Type Code

#### Table 5: SVM Performance for Arrest Type Code
Performance metrics for each Arrest Type are listed as *Precision, Recall*.

| Param | Param | Param | Param | Time | Accuracy | AUC | Felony | Misdemeanor | Infraction | Other | 
| :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- |
| gbtree | auto | 6 | 0 | | | | | | | |
| gblinear | auto | 6 | 0 | | | | | | | |
| dart | auto | 6 | 0 |  | | | | | | |
| gbtree | hist | 6 | 0 | | | | | | | |
| gblinear | hist | 6 | 0 | | | | | | | |
| dart | hist | 6 | 0 | | | | | | | |
| gbtree | hist | 5 | 0 | | | | | | | |
| gbtree | hist | 7 | 0 | | | | | | | |
| gbtree | hist | 6 | 1 | | | | | | | |
| gbtree | hist | 6 | 5 | | | | | | | |
| gbtree | hist | 6 | 10 | | | | | | | |

<a id = 'Section_7_b'></a>

### b. Descent Code

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import metrics as mt


# create variables we are more familiar with
if 'Descent Code' in df_descent1:
    yd = df_descent1['Descent Code'].values # get the labels we want
    del df_descent1['Descent Code'] # get rid of the class label
    Xd = df_descent1.values # use everything else to predict!
    
yhatd = np.zeros(yd.shape) # we will fill this with predictions

scl = StandardScaler()
Xd = scl.fit_transform(Xd)

# create cross validation iterator
cv = StratifiedKFold(n_splits=10, random_state=1234)

### XGBoost - Descent Code

In [ ]:
#KGBoost 
import pickle
import xgboost as xgb

import numpy as np
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error, classification_report, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def XGBoostd(boost, tree, depth, delta_step, etaparm):
    for train, test in cv.split(Xd,yd):
        xgb_model = xgb.XGBClassifier(booster=boost, tree_method=tree, max_depth=depth, max_delta_step=delta_step,eta=etaparm).fit(Xd[train], yd[train])
        yhatd[test] = xgb_model.predict(Xd[test])
    
    lb=LabelBinarizer()
    lb.fit(yd)
    yd_lb = lb.transform(yd)
    yhatd_lb = lb.transform(yhatd)
    print ('AUC Score', roc_auc_score(yd_lb, yhatd_lb, average='weighted'))
    
    print (mt.classification_report(yd,yhatd,digits=3))
                
    plot_class_acc(yd,yhatd,title="XGBoost")

#### Defaults

#### Modifying Booster and Tree Method Parameters

In [ ]:
%%time
if RUN_MAIN2:
    boost='gbtree'
    tree='hist'
    depth=6
    delta_step=0
    etaparm=0.3

    XGBoostd(boost,tree,depth,delta_step,etaparm)

In [ ]:
%%time
if RUN_MAIN2:
    boost='gblinear'
    tree='hist'
    depth=6
    delta_step=0
    etaparm=0.3

    XGBoostd(boost,tree,depth,delta_step,etaparm)

#### Modifying Booster, Tree Method and Max Depth Parameters

In [ ]:
%%time
if RUN_MAIN2:
    boost='gbtree'
    tree='hist'
    depth=5
    delta_step=0
    etaparm=0.3

    XGBoostd(boost,tree,depth,delta_step,etaparm)

In [ ]:
%%time
if RUN_MAIN2:
    boost='gbtree'
    tree='hist'
    depth=7
    delta_step=0
    etaparm=0.3

    XGBoostd(boost,tree,depth,delta_step,etaparm)

#### Modifying Booster, Tree Method and Max Delta Step Parameters

In [ ]:
%%time
if RUN_MAIN2:
    boost='gbtree'
    tree='hist'
    depth=6
    delta_step=1
    etaparm=0.3

    XGBoostd(boost,tree,depth,delta_step,etaparm)

In [ ]:
%%time
if RUN_MAIN2:
    boost='gbtree'
    tree='hist'
    depth=6
    delta_step=5
    etaparm=0.3

    XGBoostd(boost,tree,depth,delta_step,etaparm)

In [ ]:
%%time
if RUN_MAIN2:
    boost='gbtree'
    tree='hist'
    depth=6
    delta_step=10
    etaparm=0.3

    XGBoostd(boost,tree,depth,delta_step,etaparm)

#### XGBoost Performance for Descent Code

#### Table 6: XGBoost Performance for Descent Code
Performance metrics for each Descent Code are listed as *Precision, Recall*.

| Booster | Tree Method | Max Depth | Max Delta Step | Time | Accuracy | AUC | Black | Hispanic | White | Other | 
| :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- |
| gbtree | auto | 6 | 0 | | | | | | | |
| gblinear | auto | 6 | 0 | | | | | | | |
| dart | auto | 6 | 0 |  | | | | | | |
| gbtree | hist | 6 | 0 | | | | | | | |
| gblinear | hist | 6 | 0 | | | | | | | |
| dart | hist | 6 | 0 | | | | | | | |
| gbtree | hist | 5 | 0 | | | | | | | |
| gbtree | hist | 7 | 0 | | | | | | | |
| gbtree | hist | 6 | 1 | | | | | | | |
| gbtree | hist | 6 | 5 | | | | | | | |
| gbtree | hist | 6 | 10 | | | | | | | |

### LightGBM - Descent Code

In [ ]:
import lightgbm as lgb


def LightGBMd(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample):
    for train, test in cv.split(Xd,yd):
        lgbm_model = lgb.LGBMClassifier(n_estimators = estimators,
                                        objective = 'multiclass',
                                        learning_rate = l_rate,
                                        num_leaves = leaves,
                                        lambda_l1 = r_alpha,
                                        lambda_l2 = r_lambda, 
                                        maxdepth = depth,
                                        bagging_fraction = subsample,
                                        nthread = thread,
                                        min_gain_to_split = split_gain,
                                        feature_fraction = colsample,
                                        random_state = 17).fit(Xd[train], yd[train])
        yhatd[test] = lgbm_model.predict(Xd[test])
    
    lb = LabelBinarizer()
    lb.fit(yd)
    yd_lb = lb.transform(yd)
    yhatd_lb = lb.transform(yhatd)
    print ('AUC Score', roc_auc_score(yd_lb, yhatd_lb, average = 'weighted'))
    
    print (mt.classification_report(yd, yhatd, digits = 3))
                
    plot_class_acc(yd, yhatd, title = "LightGBM")

In [ ]:
%%time
if RUN_MAIN2:
    estimators = 100
    objective = 'multiclass'
    l_rate = 0.1
    leaves = 31
    r_alpha = 0.0
    r_lambda = 0.0
    depth = -1
    subsample = 1.0
    thread = 0
    split_gain = 0.0
    colsample = 1.0


    LightGBMd(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN2:
    estimators = 100
    objective = 'multiclass'
    l_rate = 0.1
    leaves = 31
    r_alpha = 0.0
    r_lambda = 0.0
    depth = -1
    subsample = 1.0
    thread = 0
    split_gain = 0.0
    colsample = 1.0


    LightGBMd(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN2:
    estimators = 1000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 31
    r_alpha = 0.0
    r_lambda = 0.0
    depth = -1
    subsample = 1.0
    thread = 0
    split_gain = 0.0
    colsample = 1.0


    LightGBMd(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN2:
    estimators = 10000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 31
    r_alpha = 0.0
    r_lambda = 0.0
    depth = -1
    subsample = 1.0
    thread = 0
    split_gain = 0.0
    colsample = 1.0


    LightGBMd(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN2:
    estimators = 1000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 31
    r_alpha = 0.5
    r_lambda = 0.5
    depth = 2
    subsample = 1.0
    thread = 0
    split_gain = 0.02
    colsample = 0.8


    LightGBMd(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN2:
    estimators = 10000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 31
    r_alpha = 0.5
    r_lambda = 0.5
    depth = 2
    subsample = 1.0
    thread = 0
    split_gain = 0.02
    colsample = 0.8


    LightGBMd(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN2:
    estimators = 10000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 31
    r_alpha = 0.05
    r_lambda = 0.05
    depth = 7
    subsample = 1.0
    thread = 0
    split_gain = 0.02
    colsample = 0.8


    LightGBMd(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

In [ ]:
%%time
if RUN_MAIN2:
    estimators = 10000
    objective = 'multiclass'
    l_rate = 0.05
    leaves = 34
    r_alpha = 0.05
    r_lambda = 0.05
    depth = 7
    subsample = 1.0
    thread = 0
    split_gain = 0.02
    colsample = 0.8


    LightGBMd(estimators, l_rate, leaves, r_alpha, r_lambda, depth, subsample, thread, split_gain, colsample)

#### LightGBM Performance for Descent Code

#### Table 7: LightGBM Performance for Descent Code
Performance metrics for each Descent Code are listed as *Precision, Recall*.

| Param | Param | Param | Param | Time | Accuracy | AUC | Black | Hispanic | White | Other | 
| :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- |
| gbtree | auto | 6 | 0 | | | | | | | |
| gblinear | auto | 6 | 0 | | | | | | | |
| dart | auto | 6 | 0 |  | | | | | | |
| gbtree | hist | 6 | 0 | | | | | | | |
| gblinear | hist | 6 | 0 | | | | | | | |
| dart | hist | 6 | 0 | | | | | | | |
| gbtree | hist | 5 | 0 | | | | | | | |
| gbtree | hist | 7 | 0 | | | | | | | |
| gbtree | hist | 6 | 1 | | | | | | | |
| gbtree | hist | 6 | 5 | | | | | | | |
| gbtree | hist | 6 | 10 | | | | | | | |

### Random Forest - Descent Code

In [ ]:
%%time
if RUN_RF3:
    from sklearn.ensemble import RandomForestClassifier

    maxAccPCARF = 0
    optimalEstimatorsPCARF = 0
    optimalYPCARF = []
    optimalYhatPCARF = []
    clReport = ''

    for i in range(100, 201, 50):
        clf_pipe = Pipeline(
            [('PCA',PCA(n_components=50, svd_solver='randomized')),
             ('CLF',RandomForestClassifier(n_estimators=i, n_jobs=-1))]
        )

        # now iterate through and get predictions, saved to the correct row in yhat
        for train, test in cv.split(Xd,yd):
            clf_pipe.fit(Xd[train],yd[train])
            yhatd[test] = clf_pipe.predict(Xd[test])

        plt.title('Confusion Matrix for {0} Estimators'.format(i), fontsize = 20)
        plt.imshow(mt.confusion_matrix(yd, yhatd),cmap=plt.get_cmap('Greens'),aspect='auto')
        plt.grid(False)
        plt.show()

        total_accuracy = mt.accuracy_score(yd, yhatd)
        if total_accuracy > maxAccPCARF:
            maxAccPCARF = total_accuracy
            optimalEstimatorsPCARF = i
            optimalYPCARF = yd
            optimalYhatPCARF = yhatd
            clReport = mt.classification_report(yd,yhatd,digits=3)

        lb = LabelBinarizer()
        lb.fit(yd)
        yd_lb = lb.transform(yd)
        yhatd_lb = lb.transform(yhatd)
        print ("Acc: ", total_accuracy)
        print ("AUC: ", roc_auc_score(yd_lb, yhatd_lb))

    print ('Best accuracy is ', maxAccPCARF, ' with ', optimalEstimatorsPCARF, ' estimators in a random forest with PCA')
    print(clReport)
    plot_class_acc(optimalYPCARF,optimalYhatPCARF,title="Random Forest + PCA")

In [ ]:
%%time
if RUN_RF4:
    from sklearn.ensemble import RandomForestClassifier
    maxAccRF = 0
    optimalEstimatorsRF = 0
    clReport = ''

    for i in range(100, 201, 50):
        clf = RandomForestClassifier(n_estimators=i, n_jobs=-1, oob_score=True)

        # now iterate through and get predictions, saved to the correct row in yhat
        for train, test in cv.split(Xd,yd):
            clf.fit(Xd[train],yd[train])
            yhatd[test] = clf.predict(Xd[test])

        total_accuracy = mt.accuracy_score(yd, yhatd)

        plt.title('Confusion Matrix for {0} estimators'.format(i), fontsize = 20)
        plt.imshow(mt.confusion_matrix(yd, yhatd),cmap=plt.get_cmap('Greens'),aspect='auto')
        plt.grid(False)
        plt.show()

        if total_accuracy > maxAccRF:
            maxAccRF = total_accuracy
            optimalEstimatorsRF = i
            optimalYRF = yd
            optimalYhatRF = yhatd
            clReport = mt.classification_report(yd,yhatd,digits=3)

        lb = LabelBinarizer()
        lb.fit(yd)
        yd_lb = lb.transform(yd)
        yhatd_lb = lb.transform(yhatd)
        print ("Acc: ", total_accuracy)
        print ("AUC: ", roc_auc_score(yd_lb, yhatd_lb))

    print ('Best accuracy is ', maxAccRF, ' with ', optimalEstimatorsRF, ' Estimators in a Raw Random Forest')
    print(clReport)
    plot_class_acc(optimalYRF,optimalYhatRF,title="Random Forest, Raw")

#### Table 8: Random Forest Performance for Descent Code
Performance metrics for each Descent Code are listed as *Precision, Recall*.

| Param | Param | Param | Param | Time | Accuracy | AUC | Black | Hispanic | White | Other | 
| :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- |
| gbtree | auto | 6 | 0 | | | | | | | |
| gblinear | auto | 6 | 0 | | | | | | | |
| dart | auto | 6 | 0 |  | | | | | | |
| gbtree | hist | 6 | 0 | | | | | | | |
| gblinear | hist | 6 | 0 | | | | | | | |
| dart | hist | 6 | 0 | | | | | | | |
| gbtree | hist | 5 | 0 | | | | | | | |
| gbtree | hist | 7 | 0 | | | | | | | |
| gbtree | hist | 6 | 1 | | | | | | | |
| gbtree | hist | 6 | 5 | | | | | | | |
| gbtree | hist | 6 | 10 | | | | | | | |

### SVM - Descent Code

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import metrics as mt


# create variables we are more familiar with
if 'Descent Code' in df_descent2:
    ydS = df_descent2['Descent Code'].values # get the labels we want
    del df_descent2['Descent Code'] # get rid of the class label
    XdS = df_descent2.values # use everything else to predict!
    
yhatdS = np.zeros(ydS.shape) # we will fill this with predictions

scl = StandardScaler()
XdS = scl.fit_transform(XdS)

# create cross validation iterator
cv = StratifiedKFold(n_splits=10, random_state=1234)

In [ ]:
def SVMd(krnl):
    for train, test in cv.split(XdS,ydS):
        svm_clf = SVC(C=0.5, kernel=krnl, degree=3, gamma='auto').fit(XdS[train], ydS[train])
        yhatdS[test] = svm_clf.predict(XdS[test])

    lb=LabelBinarizer()
    lb.fit(ydS)
    ydS_lb = lb.transform(ydS)
    yhatdS_lb = lb.transform(yhatdS)
    print ('AUC Score', roc_auc_score(ydS_lb, yhatdS_lb, average='weighted'))

    print (mt.classification_report(ydS,yhatdS,digits=3))

    plot_class_acc(ydS,yhatdS,title="SVM")

#### SVM Sigmoid Kernel

In [ ]:
%%time

if RUN_SVM5:
    krnl='sigmoid'
    SVMd(krnl)

#### SVM Linear Kernel

In [ ]:
%%time

if RUN_SVM6:
    krnl='linear'
    SVMd(krnl)

#### SVM Polynomial Kernel

In [ ]:
%%time

if RUN_SVM7:
    krnl='poly'
    SVMd(krnl)

#### SVM Radial Basis Function Kernel

In [ ]:
%%time

if RUN_SVM8:
    krnl='rbf'
    SVMd(krnl)

#### Stochastic Gradient Descent

In [ ]:
%%time
# Linear SVM classifier with Stochastic Descent
if RUN_MAIN2:
    from sklearn.linear_model import SGDClassifier

    regularize_const = 0.1
    iterations = 5


    for train, test in cv.split(Xd,yd):
        svm_clf = svm_sgd = SGDClassifier(alpha=regularize_const,
            fit_intercept=True, l1_ratio=0.0, learning_rate='optimal',
            loss='hinge', n_iter_no_change=iterations, n_jobs=-1, penalty='l2').fit(Xd[train], yd[train])
        yhata[test] = svm_clf.predict(Xd[test])

    lb=LabelBinarizer()
    lb.fit(yd)
    yd_lb = lb.transform(yd)
    yhatd_lb = lb.transform(yhatd)
    print ('AUC Score', roc_auc_score(yd_lb, yhatd_lb, average='weighted'))

    print (mt.classification_report(yd,yhatd,digits=3))

    plot_class_acc(yd,yhatd,title="SGD")

#### Table 9: SVM Performance for Descent Code
Performance metrics for each Descent Code are listed as *Precision, Recall*.

| Param | Param | Param | Param | Time | Accuracy | AUC | Felony | Misdemeanor | Infraction | Other | 
| :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- |
| gbtree | auto | 6 | 0 | | | | | | | |
| gblinear | auto | 6 | 0 | | | | | | | |
| dart | auto | 6 | 0 |  | | | | | | |
| gbtree | hist | 6 | 0 | | | | | | | |
| gblinear | hist | 6 | 0 | | | | | | | |
| dart | hist | 6 | 0 | | | | | | | |
| gbtree | hist | 5 | 0 | | | | | | | |
| gbtree | hist | 7 | 0 | | | | | | | |
| gbtree | hist | 6 | 1 | | | | | | | |
| gbtree | hist | 6 | 5 | | | | | | | |
| gbtree | hist | 6 | 10 | | | | | | | |

<a id = 'Section_8'></a>

## 8. Modeling and Evaluation 4

*(10 points)*

*Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.*

<a id = 'Section_9'></a>

## 9. Modeling and Evaluation 5

*(10 points)*

*Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.*

<a id = 'Section_10'></a>

## 10. Modeling and Evaluation 6

*(10 points)*

*Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.*

<a id = 'Section_11'></a>

## 11. Deployment

*(5 points)*

*How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?*

<a id = 'Section_12'></a>

## 12. Exceptional Work

*(10 points)*

*You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?*

#### Grid Searching

In [ ]:
%%time
if RUN_GRIDSEARCH:
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import GridSearchCV

    #knn example
    #create parameter grid
    knn = KNeighborsClassifier(n_neighbors = 1)

    k_range = list(range(1,32,2))
    param_grid = dict(n_neighbors = k_range)

    grid = GridSearchCV(knn, param_grid, cv = 10, scoring = 'accuracy', return_train_score = False, n_jobs)

    grid.fit(Xa, ya)

In [ ]:
if RUN_GRIDSEARCH:
    pd.DataFrame(grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']]

Grid search says .....

In [ ]:
#plt.plot(k_range, )
if RUN_GRIDSEARCH:
    score = pd.DataFrame(grid.cv_results_)['mean_test_score']
    plt.plot(k_range, score)
    plt.xlabel('Value of K for KNN')
    plt.ylabel('Mean Score Accuracy')

Multiple parameter grid search

passes dict of possible variables to run function (knn), can add more? limited value depending on the type of analysis being performed

In [ ]:
#multi parameter grid search

if RUN_GRIDSEARCH:
    k_range = list(range(1, 31, 2))
    weight_options = ['uniform', 'distance']

    param_grid = dict(n_neighbors = k_range, weights = weight_options)
    print(param_grid)

In [ ]:
%%time
if RUN_GRIDSEARCH:
    grid = GridSearchCV(knn, param_grid, cv = 10, scoring = 'accuracy', return_train_score = False)
    grid.fit(Xa, ya)
    results = pd.DataFrame(grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']]
    results

In [ ]:
#score_dist = pd.DataFrame(grid.cv_results_)['mean_test_score']

if RUN_GRIDSEARCH:
    weight_u = pd.DataFrame(grid.cv_results_)[pd.DataFrame(grid.cv_results_)['param_weights'] == 'uniform']#results from uniform parameter weights
    weight_d = pd.DataFrame(grid.cv_results_)[pd.DataFrame(grid.cv_results_)['param_weights'] == 'distance']

    plt.plot(k_range, weight_u['mean_test_score'], color = 'orange')
    plt.plot(k_range, weight_d['mean_test_score'], color = 'blue')
    plt.xlabel('Value of K for KNN')
    plt.ylabel('Mean Score Accuracy')
    plt.legend()

In [ ]:
runTime = time.time() - startTime
seconds = int(abs(runTime % 60))
allMinutes = int(abs(runTime / 60))
minutes = int(abs(allMinutes % 60))
allHours = int(abs(allMinutes / 60))
hours = int(abs(allMinutes / 60))

print ("Notebook run time: %2d:%2d:%2d" % (hours, minutes, seconds))